# Let's try KNN and see 

### First lets check all the cubic, reno and bbr graphs with different things 

In [ ]:
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics


In [ ]:
names = []
variants=["reno"]
postdelays=[20,25,30,35,40]
predelays=[20,25,30,35,40]
linkspeeds=[1000]

In [ ]:
for v in variants :
    for pred in predelays:
        for pod in postdelays:
            for ls in linkspeeds :
                names.append(v+"-"+str(pred)+"-"+str(pod)+"-"+str(ls))
print(names)

In [ ]:
cubic_files

In [ ]:
reno_files

In [ ]:
bbr_files 

In [ ]:
ts = []

In [ ]:
for f in bbr_files :
    fig, ax = plt.subplots()
    time,data = plot_bif(f)
    ts.append(data)

In [ ]:
from tslearn.utils import to_time_series_dataset
X = to_time_series_dataset(ts)

In [ ]:
!pip install numpy==1.21.6

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
model = TimeSeriesKMeans(n_clusters=3, metric="softdtw", max_iter=10)
labels = model.fit(X)

# Getting parameters from the graph for decision trees. 

### The cwnd decrease, the cwnd increase. 

In [ ]:
f = "cubic"

In [ ]:
def plot_bif(f):
    fig, ax = plt.subplots(1,1, figsize=(15,8))
#     get_bif(ax,f)
#     get_cwnd(ax,f)
#     get_ma(ax,f)
#     get_ema(ax,f)
#     get_wwlt(ax,f)
    time, data = get_fft_smoothening(ax,f)
#     ax.title.set_text(f)
    ax.legend()
    plt.show()
#     fig, ax = plt.subplots(1,1, figsize=(15,8))
#     ax.plot(new_time[5:], grad[5:], color='r', lw=0.8)
#     plt.show()
    return time, data

In [ ]:
time, data = plot_bif("cubic")

### Using 1 RTT as the window size to calculate maximum and minimum

In [ ]:
data[1], time[2]

In [ ]:
rtt = 0.1

In [ ]:
from collections import Counter
diff = []
for i in range(1,len(time)):
    curr = round(time[i]-time[i-1],3)
    diff.append(curr)
Counter(diff)

In [ ]:
step = 0.002

In [ ]:
from sortedcontainers import SortedDict
from collections import OrderedDict

In [ ]:
def get_grad(dt):
    first = next(iter(dt))
    last = next(reversed(dt))
    time_first = dt[first][0]
    time_last = dt[last][0]
    grad = (last-first)/(time_last-time_first)
    return grad, (first, last, time_first, time_last)

In [ ]:
left = 0
right = 0
grad = []
grad_meta = []
# dt = SortedDict()
dt = OrderedDict()
# continue loop till we reach the end of the right
while right < len(time) and left < len(time):
    # choose a right and while choosing get max and min
    while right < len(time) and time[right] - time[left] < rtt :
        if data[right] not in dt :
            dt[data[right]] = list()
        dt[data[right]].append(time[right])
        right+=1
    # save gradient and the data and time period with it
    curr_grad, curr_meta = get_grad(dt) 
    grad.append(curr_grad)
    grad_meta.append(curr_meta)
    # choose the next left
    prev_left = left 
    while left < len(time) and time[left] - time[prev_left] < step :
        if data[left] in dt :
            dt[data[left]].pop(0)
            if len(dt[data[left]])==0:
                del dt[data[left]]
        left+=1


In [ ]:
len(grad), len(time), len(data)

In [ ]:
new_time = []
for i in range(len(grad)):
    t = (grad_meta[i][3] + grad_meta[i][2])/2
    new_time.append(t)

In [ ]:
len(new_time)

In [ ]:
color = []
i = 0
for t in range(len(new_time)):
    while i < len(time) and time[i]<new_time[t]:
        if(grad[t]>0):
            color.append(1)
        else :
            color.append(0)
        i+=1
while i < len(time) :
    if(grad[len(new_time)-1]>0):
        color.append(1)
    else :
        color.append(0)
    i+=1

In [ ]:
len(color)

In [ ]:
pos = []
neg = []
for i in range(len(data)):
    if color[i]==1:
        pos.append(data[i])
        neg.append(0)
    else :
        pos.append(0)
        neg.append(data[i])

In [ ]:
len(pos), len(neg)

In [ ]:
len(time)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,8))
time, data = get_fft_smoothening(ax,f)
ax.scatter(time,pos, color='r')
ax.scatter(time,neg,color='b')
plt.show()

In [ ]:
len(time)

In [ ]:
plt.plot(grad)

# Plotting the sites BIF by applying the filter

In [ ]:
sites = "google youtube facebook twitter wikipedia amazon instagram linkedin reddit whatsapp openai yahoo bing xvideos live pornhub microsoft vk"

In [ ]:
sites = sites.split(" ")

In [ ]:
def plot_bif(files):
    for f in files:
        fig, ax = plt.subplots(1,1, figsize=(15,8))
        time, data = get_bif(ax,f)
        time, data = get_fft_smoothening(ax,f)
        plt.show()

In [ ]:
plot_bif(sites)

# Calculating the DTW of CWND with all filtered and unfiltered BIF

In [ ]:
from datetime import datetime

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
from dtw import dtw

In [ ]:
f = "cubic"
ds = []
labels = []
fig, ax = plt.subplots(1,1, figsize=(15,8))
time_cwnd, data_cwnd = get_cwnd(ax,f)
time, data = get_bif(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("BIF Error ", distance)
ds.append(distance)
labels.append("BIF")

time, data = get_ma(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("MA Error ", distance)
ds.append(distance)
labels.append("MA")

time, data = get_ema(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("EMA Error ", distance)
ds.append(distance)
labels.append("EMA")


time, data = get_fft_smoothening(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("FFT Error ", distance)
ds.append(distance)
labels.append("FFT")
ax.title.set_text(f)
ax.legend()
plt.show()

plt.bar(labels, ds, color ='maroon',width = 0.4)
plt.show()

In [ ]:
def plot_one_dtw(files):
    for f in files:
        fig, ax = plt.subplots(1,1, figsize=(15,8))
        time_cwnd, data_cwnd = get_cwnd(ax,f)
        time, data = get_bif(ax,f)
        start_time = datetime.now()
        distance, path = fastdtw(data_cwnd, data, dist=euclidean)
        end_time = datetime.now()
        print("Time taken in seconds", end_time - start_time)
        print("The error : ", distance)
#         time, data = get_ma(ax,f)
#         time, data = get_ema(ax,f)
#         time, data = get_fft_smoothening(ax,f)
        ax.title.set_text(f)
        ax.legend()
        plt.show()
        

In [ ]:
f = "bbr"
ds = []
labels = []
fig, ax = plt.subplots(1,1, figsize=(15,8))
time_cwnd, data_cwnd = get_cwnd(ax,f)
time, data = get_bif(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("BIF Error ", distance)
ds.append(distance)
labels.append("BIF")

time, data = get_ma(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("MA Error ", distance)
ds.append(distance)
labels.append("MA")

time, data = get_ema(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("EMA Error ", distance)
ds.append(distance)
labels.append("EMA")


time, data = get_fft_smoothening(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("FFT Error ", distance)
ds.append(distance)
labels.append("FFT")
ax.title.set_text(f)
ax.legend()
plt.show()

plt.bar(labels, ds, color ='maroon',width = 0.4)
plt.show()

In [ ]:
f = "cubic"
ds = []
labels = []
fig, ax = plt.subplots(1,1, figsize=(15,8))
time_cwnd, data_cwnd = get_cwnd(ax,f)
time, data = get_bif(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("BIF Error ", distance)
ds.append(distance)
labels.append("BIF")

time, data = get_ma(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("MA Error ", distance)
ds.append(distance)
labels.append("MA")

time, data = get_ema(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("EMA Error ", distance)
ds.append(distance)
labels.append("EMA")


time, data = get_fft_smoothening(ax,f)
distance, path = fastdtw(data_cwnd, data, dist=euclidean)
print("FFT Error ", distance)
ds.append(distance)
labels.append("FFT")
ax.title.set_text(f)
ax.legend()
plt.show()

plt.bar(labels, ds, color ='maroon',width = 0.4)
plt.show()

In [ ]:
def get_window(f,p):
    algo_cc = f
    flows = process_flows(algo_cc, "./Nebby/measurements/")
    data = []
    time = []
    for port in flows.keys():
        if flows[port]["last_ack"] == 0 :
            continue
        else :
            data = flows[port]['windows']
            time = flows[port]['times']
            if p == "p":
                for i in range(len(flows[port]['windows'])):
                    print("{0:10}{1:10}".format(flows[port]['times'][i], flows[port]['windows'][i]))
        time_index = len(time)-1
        for index in range(len(flows[port]['times'])-1):
            if flows[port]['times'][index+1] - flows[port]['times'][index] > 1 :
                time_index = index
        time_last = flows[port]['times'][time_index]
        data = data[:time_index+1]
        time = time[:time_index+1]
        return data, time

In [ ]:
ig, ax = plt.subplots(figsize=(14, 10))
x = data_cwnd
y = data
# Remove the border and axes ticks
fig.patch.set_visible(False)
ax.axis('off')

for [map_x, map_y] in path:
    ax.plot([map_x, map_y], [x[map_x], y[map_y]], '--k', linewidth=4)

ax.plot(x, '-ro', label='x', linewidth=0.5, markersize=1, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
ax.plot(y, '-bo', label='y', linewidth=4, markersize=1, markerfacecolor='skyblue', markeredgecolor='skyblue')
ax.set_title("DTW Distance", fontsize=28, fontweight="bold")

In [ ]:
ig, ax = plt.subplots(figsize=(14, 10))

# Remove the border and axes ticks
fig.patch.set_visible(False)
ax.axis('off')

for [map_x, map_y] in warp_path:
    ax.plot([map_x, map_y], [x[map_x], y[map_y]], '--k', linewidth=4)

ax.plot(x, '-ro', label='x', linewidth=4, markersize=20, markerfacecolor='lightcoral', markeredgecolor='lightcoral')
ax.plot(y, '-bo', label='y', linewidth=4, markersize=20, markerfacecolor='skyblue', markeredgecolor='skyblue')
ax.set_title("DTW Distance", fontsize=28, fontweight="bold")

# Plotting the BIF for all the algos : 1000Kbps LINKSPEED and 50ms POST DELAY 

In [ ]:
def lowpassfilter(signal, thresh=0.63, wavelet="db4"):
    thresh = thresh*np.nanmax(signal)
    coeff = pywt.wavedec(signal, wavelet, mode="per")
    coeff[1:] = (pywt.threshold(i, value=thresh, mode="hard") for i in coeff[1:])
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode="per")
    return reconstructed_signal
# def plot_dwt(ax,signal,thresh=0.63, wv="db4"):
#     fig, ax = plt.subplots(figsize=(12,8))
#     rec = lowpassfilter(signal, thresh, wv)
#     ax.plot(rec, 'k', label='DWT smoothing}', linewidth=2)
#     ax.set_title('Removing High Frequency Noise with DWT', fontsize=18)
#     ax.set_ylabel('Signal Amplitude', fontsize=16)
#     ax.set_xlabel('Sample No', fontsize=16)
#     plt.show()
# plot_dwt(0.1,"rbio1.5")

In [ ]:
import pandas as pd
import pywt

In [ ]:
def get_algo_files(name=[]):
    if len(name) == 0:
        ccs = "reno cubic bbr veno bic cdg dctcp highspeed htcp hybla illinois lp nv scalable vegas westwood yeah"
        ccs = ccs.split(" ")
    else :
        ccs = name
    return ccs

In [ ]:
def plot_cwnd_bf(files):
    path = "/home/lakshayr/nus_intern/Nebby/custom_clients/statsfiles"
    for f in files:
        algo_cc = f
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        port = list(flows.keys())[0]
        
        statsfile = path + "/" + f + "-stats.csv"
        df_c = pd.read_csv(statsfile,sep=",")
        start = df_c['ts'][1]
        df_c['ts'] = df_c['ts'] - start
        cwnd_bytes = df_c['tcpi_snd_cwnd'].mul(df_c['snd'])
        
        fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,5))
        ax1.plot(flows[port]['times'][1:int(len(flows[port]['times']))], flows[port]['windows'][1:int(len(flows[port]['times']))], color='b',lw='0.5')
        ax1.title.set_text(f)
        ax2.plot(df_c['ts'][1:], cwnd_bytes[1:])
        plt.show()

In [ ]:
def get_wwlt(ax,f):
    data, time = get_window(f, "n")
    thresh = 0.4
    wv = "rbio1.5"
    rec = lowpassfilter(data, thresh, wv)
    start = len(rec) - len(time)
    ax.plot(time,rec[start:], label='DWT smoothing}', linewidth=2)
    return time, data
def get_bif(ax,f):
    algo_cc = f
    data, time = get_window(f, "n")
#     flows = process_flows(algo_cc, "./Nebby/measurements/")
#     port = list(flows.keys())[0]
#     start_bif = flows[port]['times'][0]
#     ax.plot(flows[port]['times'][1:int(len(flows[port]['times']))], flows[port]['windows'][1:int(len(flows[port]['windows']))], label="bif",color='b',lw=0.5,alpha=0.5)
    ax.plot(time, data,label="bif",color='b',lw=0.5,alpha=0.5)
    return time,data
    
def get_cwnd(ax,f):
    path = "/home/lakshayr/nus_intern/Nebby/custom_clients/statsfiles"
    statsfile = path + "/" + f + "-stats.csv"
    df_c = pd.read_csv(statsfile,sep=",")
    start_cwnd = df_c['ts'][1]

    fix = 0.1
    if(f=="reno"):
        fix=0.2
    df_c['ts'] = df_c['ts'] - start_cwnd  + fix
    cwnd_bytes = df_c['tcpi_snd_cwnd'].mul(df_c['snd'])
    cwnd_times = df_c['ts']
    data, time = get_window(f, "n")
    time_thresh = time[-1]
    cwnd_thresh = len(time)
    for t in range(len(df_c['ts'])):
        if df_c['ts'][t] > time_thresh:
            cwnd_thresh = t+1
            break
    ax.plot(cwnd_times[1:cwnd_thresh], cwnd_bytes[1:cwnd_thresh], label="cwnd",color='r', lw=1.5, alpha=1)
    return list(cwnd_times[1:cwnd_thresh]), list(cwnd_bytes[1:cwnd_thresh])

def get_ma(ax,f):
    data, time = get_window(f, "n")
    ma_data = []
    k = 20
    for i in range(len(data)):
        if i-k < 0:
            curr = sum(data[0:i+1])
            curr += sum(data[i+1:abs(i-k)])
            curr = float(curr)/k
        else :
            curr = sum(data[i-k:i+1]);
            curr = float(curr)/k
        ma_data.append(curr)
    ax.plot(time, ma_data, color='g', label="MA", lw=1.5, alpha=1)
    return time, ma_data
    
def get_ema(ax,f):
    data, time = get_window(f, "n")
    ma_data = []
    alpha=0.2
    for i in range(len(data)):
        if i == 0:
            curr = data[i]
        else :
            curr = alpha*data[i] + (1-alpha)*ma_data[i-1]
        ma_data.append(curr)
    ax.plot(time, ma_data, color='m', label="EMA")
    return time, ma_data
    
def get_fft_smoothening(ax,f):
    rtt=0.1
    data, time = get_window(f, "n")
    print(len(data), " ", len(time))

    yf, xf = get_fft(data)
    abs_yf = np.abs(yf)
#     print(len(abs_yf), len(xf))
    sort_abs_yf = sorted(abs_yf)
    
#     plt.title("FFT Decomposition")
#     plt.plot(xf, sort_abs_yf, color='b', alpha=0.5, lw=1)
#     plt.scatter(xf, sort_abs_yf, color='r', s=2)
#     plt.show()

    thresh  = (1/rtt)
    print("RTT", rtt, "Freq ", thresh)
#     print(xf)
    thresh_ind = 0
    for i in range(len(xf)) :
        freq = xf[i]
        if(freq > thresh):
            thresh_ind = i
            break
#     print("Index ", thresh_ind)
#     print(xf[0:thresh_ind])
#     plt.title("Cleaned Frequencies")
#     plt.plot(xf[0:thresh_ind],np.abs(yf[0:thresh_ind]))
#     plt.show()

    yf_clean = yf
    yf_clean[thresh_ind+1:] = 0
    new_f_clean = irfft(yf_clean)
#     plt.title("Frequencies smaller than " + str(thresh))
#     plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    start_len = len(time) - len(new_f_clean)
    ax.plot(time[start_len:], new_f_clean, 'k', label='FFT smoothening', linewidth=1.5)
#     plt.legend()
#     plt.show()
    return time[start_len:], new_f_clean


In [ ]:
def plot_fft_cwnd(files):
    for f in files :
        print(f)
        path = "/home/lakshayr/nus_intern/Nebby/custom_clients/statsfiles"
        statsfile = path + "/" + f + "-stats.csv"
        df_c = pd.read_csv(statsfile,sep=",")
        start_cwnd = df_c['ts'][1]
        
        fix = 0.1
        if(f=="reno"):
            fix=0.2
        df_c['ts'] = df_c['ts'] - start_cwnd  + fix
        time = df_c['ts'][1:]
        cwnd_bytes = df_c['tcpi_snd_cwnd'].mul(df_c['snd'])

#         print(len(data), " ", len(time))
        data = cwnd_bytes[1:]
        yf, xf = get_fft(data)
        abs_yf = np.abs(yf)
#         print(len(abs_yf), len(xf))
        sort_abs_yf = sorted(abs_yf)
#         for x in range(len(xf)):
#             print(xf[x],abs_yf[x])
        plt.title("FFT Decomposition CWND")
        plt.plot(xf[1:], abs_yf[1:], color='b', alpha=0.5, lw=1)
        plt.scatter(xf[1:], abs_yf[1:], color='r', s=2)
        plt.show()
        
        quantile = np.quantile(sort_abs_yf,75)
        thresh_f = af[abs_yf.index(quantile)]
        print("Freq ", thresh_f)
        
        
        data, time = get_window(f, "n")
        print(len(data), " ", len(time))

        yf, xf = get_fft(data)
#         abs_yf = np.abs(yf)
#         plt.title("FFT Decomposition BIF")
#         plt.plot(xf, abs_yf, color='b', alpha=0.5, lw=1)
#         plt.scatter(xf, abs_yf, color='r', s=2)
#         plt.show()
        corrected_data = xf < thresh_f
        
#         plt.title("Log FFT Decomposition")
#         plt.plot(xf[1:], np.log10(abs_yf[1:]), color='b', alpha=0.5, lw=1)
#         plt.scatter(xf[1:], np.log10(abs_yf[1:]), color='r', s=2)
#         plt.show()
        # take the mean of the log and then use that 
        
#         print("Thresh = ", pow(10,thresh))
#         correct = abs_yf[1:] > pow(10,thresh)
#         corrected_yf = yf[1:] * correct
#         corrected_data = irfft(corrected_yf)
#         print(corrected_data)
#         plt.plot(time, data, color="b", alpha=0.5, label='original signal')
#         plt.plot(time[2:], corrected_data, 'k', label='FFT smoothening', linewidth=1.5)
#         plt.show()
#         
#           thresh_ind = 0
#         for i in range(len(xf)) :
#             freq = xf[i]
#             if(freq > thresh):
#                 thresh_ind = i
#                 break
#         print("Index ", thresh_ind)
#         print(xf[0:thresh_ind])
#         plt.title("Cleaned Frequencies")
#         plt.plot(xf[0:thresh_ind],np.abs(yf[0:thresh_ind]))
#         plt.show()

#         yf_clean = yf
#         yf_clean[thresh_ind+1:] = 0
#         new_f_clean = irfft(yf_clean)
# #         plt.title("Frequencies smaller than " + str(thresh))
# #         plt.plot(time, data, color="b", alpha=0.5, label='original signal')
#         start_len = len(time) - len(new_f_clean)
#         plt.plot(time[start_len:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
#         plt.legend()
#         plt.show()

In [ ]:
plot_fft_cwnd(["cubic"])

In [ ]:
def get_fft(data):
    n = len(data)
    data_step = 0.002
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    return yf,xf


In [ ]:
def plot_one(files):
    for f in files:
        fig, ax = plt.subplots(1,1, figsize=(15,8))
#         get_bif(ax,f)
        get_cwnd(ax,f)
#         get_ma(ax,f)
#         get_ema(ax,f)
#         get_wwlt(ax,f)
        get_fft_smoothening(ax,f)
        ax.title.set_text(f)
        ax.legend()
        plt.show()

In [ ]:
plot_one(ccs)

In [ ]:
plot_cwnd_bf(ccs)

In [ ]:
data, time = get_window(ccs[0], p="n")

In [ ]:
ccs = get_algo_files()
plot_g(ccs,z=1)

In [ ]:
file = folder + "/" + "cubic-stats.csv"
df = pd.read_csv(file,sep=",")
plt.title("Senders CWND for CUBIC")
start = df['ts'][1]
df['ts'] = df['ts'] - start 
plt.plot(df['ts'], df['tcpi_snd_cwnd'])
plt.show()

# Getting the CWND trace from custom client

In [ ]:
folder = "/home/lakshayr/nus_intern/Nebby/custom_clients/statsfiles"

In [ ]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk(folder):
    f.extend(filenames)

In [ ]:
file = folder + "/" + filenames[0]

In [ ]:
print(file)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(file,sep="\t")

In [ ]:
df['ts']

In [ ]:
plt.title("Senders CWND for CUBIC")
plt.plot(df['ts'], df['tcpi_snd_cwnd'])
plt.show()

# Moving Average 

### 1 - Normal Moving Average
### 2 - Exponential Moving Average

In [ ]:
curr_files = get_files("","35","500")
print(curr_files)

In [ ]:
def plot_ma(files, t=1, k=10, alpha=0.5):
    for file in files : 
        print(file)
        data, time = get_window(file, "n")
        if t == 0 or t == 1:
            ma_data = []
            for i in range(len(data)):
                if i-k < 0:
                    curr = sum(data[0:i+1])
                    curr += sum(data[i+1:abs(i-k)])
                    curr = float(curr)/k
                else :
                    curr = sum(data[i-k:i+1]);
                    curr = float(curr)/k
                ma_data.append(curr)
            title = "MA " + str(k)
            plt.figure(figsize =(10,8))
            plt.title(title)
            plt.plot(time, data, color="b", alpha=0.5, label='original signal')
            plt.plot(time, ma_data, 'k', label="MA")
            plt.show()
        if t == 0 or t == 2 :
            ma_data = []
            for i in range(len(data)):
                if i == 0:
                    curr = data[i]
                else :
                    curr = alpha*data[i] + (1-alpha)*ma_data[i-1]
                ma_data.append(curr)
            plt.figure(figsize =(10,8))
            plt.title("EMA "+str(alpha))
            plt.plot(time, data, color="b", alpha=0.5, label='original signal')
            plt.plot(time, ma_data, 'k', label="EMA")
            plt.show()

In [ ]:
plot_ma(ccs,0, 20, 0.07)

# RTT threshold

In [ ]:
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics


In [ ]:
def get_window(f,p):
    algo_cc = f
    flows = process_flows(algo_cc, "./Nebby/measurements/")
    data = []
    time = []
    for port in flows.keys():
        if flows[port]["last_ack"] == 0 :
            continue
        else :
            data = flows[port]['windows']
            time = flows[port]['times']
            if p == "p":
                for i in range(len(flows[port]['windows'])):
                    print("{0:10}{1:10}".format(flows[port]['times'][i], flows[port]['windows'][i]))
        return data, time

In [ ]:
curr_files = get_files("","35","500")
print(curr_files)

In [ ]:
def get_fft(data):
    n = len(data)
    data_step = 0.001
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    return yf,xf


In [ ]:

def plot_rtt(files):
    for file in files :
        print(file)
        rtt = 2*float(file.split("-")[1])/1000
        data, time = get_window(file, "n")

#         print(len(data), " ", len(time))

        yf, xf = get_fft(data)
        abs_yf = np.abs(yf)
#         print(len(abs_yf), len(xf))
        sort_abs_yf = sorted(abs_yf)
        plt.title("FFT Decomposition")
#         plt.plot(xf, sort_abs_yf, color='b', alpha=0.5, lw=1)
#         plt.scatter(xf, sort_abs_yf, color='r', s=2)
#         plt.show()

        thresh  = (1/rtt)
        print("RTT", rtt, "Freq ", thresh)
#         print(xf)
        thresh_ind = 0
        for i in range(len(xf)) :
            freq = xf[i]
            if(freq > thresh):
                thresh_ind = i
                break
        print("Index ", thresh_ind)
#         print(xf[0:thresh_ind])
#         plt.title("Cleaned Frequencies")
#         plt.plot(xf[0:thresh_ind],np.abs(yf[0:thresh_ind]))
#         plt.show()

        yf_clean = yf
        yf_clean[thresh_ind+1:] = 0
        new_f_clean = irfft(yf_clean)
#         plt.title("Frequencies smaller than " + str(thresh))
#         plt.plot(time, data, color="b", alpha=0.5, label='original signal')
        start_len = len(time) - len(new_f_clean)
        plt.plot(time[start_len:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
#         plt.legend()
#         plt.show()

In [ ]:
plot_rtt(curr_files)

In [ ]:
import csv
import matplotlib.pyplot as plt
import sys
import math

SHOW=True
MULTI_GRAPH=False
SMOOTHENING=False
ONLY_STATS=False
s_factor=0.9

'''
TODO: 
o Add functionality where you only plot flows that send more than x bytes of data
o Sort stats and graphs by flow size
o Organize plots by flow size (larger flows have larger graphs)
o Custom smoothening function
'''

fields=["time", "frame_time_rel", "tcp_time_rel", "frame_num", "frame_len", "ip_src", "src_port", "ip_dest", "dest_port", "tcp_len", "seq", "ack"]

class pkt:
    contents=[]
    def __init__(self, fields) -> None:
        self.contents=[]
        for f in fields:
            self.contents.append(f)

    def get(self, field):
        return self.contents[fields.index(field)]
        

def process_flows(cc, dir):
    with open(dir+cc+"-tcp.csv") as csv_file:
        print("Reading "+dir+cc+"-tcp.csv...")
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        total_bytes=0
        '''
        Flow tracking:
        o Identify all packets that are either sourced from or headed to 100.64.0.2
        o Group different flows by client's port
        '''
        flows={}
        data_sent=0
        port_set = set()
        for row in csv_reader:
            packet=pkt(row)
            validPkt=False
            if line_count==0:
                # reject the header
                line_count+=1
                continue
            if data_sent == 0 : 
                if "100.64.0." in packet.get("ip_src"):
                    num = int(packet.get("ip_src")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_src")
                if "100.64.0." in packet.get("ip_dest"):
                    num = int(packet.get("ip_dest")[-1])
                    if num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_dest")
                if data_sent == 0:
                    continue
                    
            if packet.get("ip_src")==host_port and packet.get("frame_time_rel")!='' and packet.get("ack")!='': 
                # we care about this ACK packet
                validPkt=True
                port=packet.get("src_port")
                if port not in flows:
                    flows[port]={"serverip":packet.get("ip_dest"), "serverport":packet.get("dest_port"), "act_times":[],"times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0}
                flows[port]["act_times"].append(packet.get("time"))
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                flows[port]["bif"]-=max(0,(int(packet.get("ack"))-int(flows[port]["last_ack"])))
                flows[port]["last_ack"]=max(int(flows[port]["last_ack"]),int(packet.get("ack")))
                #flows[port]["windows"].append(int(flows[port]["bif"]))
                flows[port]["pif"]-=1
                flows[port]["cwnd"].append(flows[port]["pif"])
            elif packet.get("ip_dest")==host_port and packet.get("frame_time_rel")!='' and packet.get("seq")!='':
                #we care about this Data packet
                validPkt=True
                port=packet.get("dest_port")
                if port not in flows:
                    flows[port]={"serverip":packet.get("ip_src"), "serverport":packet.get("src_port"),"act_times":[], "times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0}
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                flows[port]["act_times"].append(packet.get("time"))
                flows[port]["bif"]+=max(0,(int(packet.get("seq"))-int(flows[port]["last_seq"])))
                flows[port]["last_seq"]=max(flows[port]["last_seq"],int(packet.get("seq")))
                flows[port]["pif"]+=1
                flows[port]["cwnd"].append(flows[port]["pif"])
            if SMOOTHENING and validPkt and len(flows[port]["windows"])>2:
                flows[port]["windows"].append(int((s_factor*flows[port]["windows"][-1])+((1-s_factor)*flows[port]["bif"])))
            elif validPkt:
                flows[port]["windows"].append(int(flows[port]["bif"]))
            line_count+=1
            total_bytes+=int(packet.get("frame_len"))
            #print(line_count, total_bytes)
            
        print("total bytes processed:", total_bytes/1000, "KBytes for", cc, "(unlimited)")
    return flows

def custom_smooth_function():
    pass

def get_flow_stats(flows):
    num=len(flows.keys())
    print("FLOW STATISTICS: \nNumber of flows: ", num)
    print("------------------------------------------------------------------------------")
    print('%6s'%"port", '%15s'%"SrcIP", '%8s'%"SrcPort",  '%8s'%"duration",  '%8s'%"start",  '%8s'%"end", '%8s'%"Sent (B)", '%8s'%"Recv (B)",)
    for k in flows.keys():
        print('%6s'%k, '%15s'%flows[k]["serverip"], '%8s'%flows[k]["serverport"], '%8s'%str('%.2f'%(flows[k]["times"][-1]-flows[k]["times"][0])), '%8s'%str('%.2f'%flows[k]["times"][0]), '%8s'%str('%.2f'%flows[k]["times"][-1]), '%8s'%flows[k]["last_seq"], '%8s'%flows[k]["last_ack"])
        #print("    * Flow "+str(k)+": ", flows[k]["last_ack"], " ", flows[k]["last_seq"], " bytes transfered.")
    return num

def run(files):
    flows = {}
    for f in files:
        algo_cc=f
        #Get the data for all the flows
        print("==============================================================================")
        print("opening trace ../measurements/"+algo_cc+".csv...")
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        #decide on final graph layout
        num = get_flow_stats(flows)

        if ONLY_STATS:
            sys.exit()

        if num==1:
            MULTI_GRAPH=False
        #grid size
        if MULTI_GRAPH:
            size=(0,0)
            grids={1:(2,2), 2:(2,2), 4:(2,2), 6:(2,3), 9:(3,3), 12:(3,4), 15:(3,5), 16:(4,4), 20:(5,4), 24:(6,4), 30:(6,5), 36:(6,6), 40:(8,5), 42:(8,7), 49:(7,7)}
            g=num
            while g<=49 and g not in grids:
                g+=1
            if g in grids.keys():
                size=grids[g]
            else:
                size=grids[49]  
            fig, axs = plt.subplots(size[0], size[1])
            for i in range(size[0]):
                for j in range(size[1]):
                    #axs[i][j].legend(loc="lower right")
                    if i==size[0]-1:
                        axs[i][j].set_xlabel("Time (s)")
                    if j==0:
                        axs[i][j].set_ylabel("Bytes in flight")
        else:
            plt.xlabel("Time (s)")
            plt.ylabel("Bytes in flight")
        counter=0
        for port in flows.keys():
            if MULTI_GRAPH:  
                axs[counter%size[0]][(counter//size[0])%size[1]].scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
                axs[counter%size[0]][(counter//size[0])%size[1]].plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
            else:
                plt.plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
                plt.scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
            counter+=1
        if MULTI_GRAPH:
            counter=0
            for port in flows.keys():
                axs[counter%size[0]][(counter//size[0])%size[1]].legend()
                counter+=1
        else:
            plt.legend()
        if MULTI_GRAPH:
            fig.set_size_inches(16, 12)
        if SHOW:
            plt.show()
        else:
            plt.savefig("../logs/results/"+algo_cc+".png", dpi=600, bbox_inches='tight', pad_inches=0)
    return flows


In [ ]:
flows = run(["test15"])

In [ ]:
def smoothen(data, func, para=10):
    times = data['times']
    windows = data['windows']
    new_times = []
    new_windows =[]
    if func == "cma" :
        for t in range(len(times)):
            # Keeping all the data points
            win = int(para/2)
            if t < win :
                val = sum(windows[0:t]) + sum(windows[t:t+win])
            else :
                val = sum(windows[t-win:t]) + sum(windows[t:t+win])
            val = val/para
            new_times.append(times[t])
            new_windows.append(val)

    return new_times, new_windows
                    

In [ ]:
def plot_nice(files):
    for f in files:
        algo_cc = f
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        for port in flows.keys():
            smooth = ""
            times, windows = smoothen(flows[port], "cma", 20) 
            fig, (ax1,ax2) = plt.subplots(2,1, figsize=(10,10))
            title = f + " : " + str(port) + smooth
            ax1.plot(flows[port]['times'], flows[port]['windows'], color='b',lw='0.5')
            ax1.title.set_text(title)
            ax2.plot(times, windows,color='r',lw='0.5')
            ax2.title.set_text(title+"_smooth")
#             plt.savefig(title + ".png")
            plt.show()
lags=[15,25,35,45]
files = []
for l in lags :
    name = "test"+str(l)
    files.append(name)


# Working with graphs

In [ ]:
post_delays = ["15","25","35","45"]
link_speeds = ["100", "200", "500", "1000"]
ccs = ["cubic", "reno", "bbr"]

In [ ]:
files = []
for cc in ccs :
    for d in post_delays:
        for link_speed in link_speeds :
            files.append(cc+"-"+d+"-"+link_speed)

In [ ]:
def get_files(cc="",d="",s=""):
    new_f = []
    cc_list = [cc]
    d_list = [d]
    s_list = [s]
    if cc == "" :
        cc_list = ccs
    if d == "" :
        d_list = post_delays
    if s == "" :
        s_list = link_speeds
    for de in d_list :
        for sp in s_list:
            for co in cc_list:
                new_f.append(co+"-"+de+"-"+sp)
    return new_f

In [ ]:
import pprint

In [ ]:
def get_window(f,p):
    algo_cc = f
    flows = process_flows(algo_cc, "./Nebby/measurements/")
    data = []
    time = []
    for port in flows.keys():
        if flows[port]["last_ack"] == 0 :
            continue
        else :
            data = flows[port]['windows']
            time = flows[port]['times']
            if p == "p":
                for i in range(len(flows[port]['windows'])):
                    print("{0:10}{1:10}".format(flows[port]['times'][i], flows[port]['windows'][i]))
        time_index = len(time)-1
        for index in range(len(flows[port]['times'])-1):
            if flows[port]['times'][index+1] - flows[port]['times'][index] > 1 :
                time_index = index
        time_last = flows[port]['times'][time_index]
        data = data[:time_index+1]
        time = time[:time_index+1]
        return data, time

In [ ]:
def plot_g(files,z=1):
    for f in files:
        algo_cc = f
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        for port in flows.keys():
            if flows[port]["last_ack"] == 0 :
                continue
            smooth = ""
#             times, windows = smoothen(flows[port], "cma", 20) 
#             fig, (ax1,ax2) = plt.subplots(2,1, figsize=(10,10))
#             title = f + " : " + str(port) + smooth
#             ax1.plot(flows[port]['times'], flows[port]['windows'], color='b',lw='0.5')
#             ax1.title.set_text(title)
#             ax2.plot(times, windows,color='r',lw='0.5')
#             ax2.title.set_text(title+"_smooth")
            fig, ax, = plt.subplots(1,1, figsize=(20,10))
            ax.plot(flows[port]['times'][1:int(len(flows[port]['times'])/z)], flows[port]['windows'][1:int(len(flows[port]['times'])/z)], color='b',lw='0.5')
#             ax.scatter(flows[port]['times'], flows[port]['windows'], color='r', s=5)
            ax.title.set_text(f)
#             plt.savefig(title + ".png")
            plt.show()

In [ ]:
curr_files = get_files("cubic","15","500")
get_window(curr_files)

In [ ]:
curr_files = get_files("cubic","15","100")
plot_g(curr_files,z=10)

In [ ]:
curr_files = get_files("bbr","","1000")
plot_g(curr_files)

### CC Same, PD diff, LS Same
##### 100 and 200 are pretty low, checking on 500 and 1000

In [ ]:
link_speeds = ["500", "1000"]

In [ ]:
curr_files = get_files("reno","","500")
plot_g(curr_files)

In [ ]:
curr_files = get_files("cubic","","500")
plot_g(curr_files)

In [ ]:
curr_files = get_files("bbr","","500")
plot_g(curr_files)

### CC Same, PD same, LS diff 

In [ ]:
curr_files = get_files("cubic","35","")
plot_g(curr_files)

In [ ]:
curr_files = get_files("bbr","35","")
plot_g(curr_files)

In [ ]:
curr_files = get_files("reno","35","")
plot_g(curr_files)

### Zooming in 

In [ ]:
curr_files = get_files("cubic","","100")
plot_g(curr_files,z=10)

In [ ]:
curr_files = get_files("bbr","","100")
plot_g(curr_files,z=10)

### CC Diff, PD 35ms, LS 500ms


In [ ]:
curr_files = get_files("","35","500")
plot_g(curr_files)

## Running different filters on it.

In [ ]:
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics
curr_files = get_files("cubic","35","500")
data, time = get_window(curr_files[0], "n")
def get_fft(data):
    n = len(data)
    data_step = 0.00001
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    plt.plot(xf, np.abs(yf))
    return yf,xf
yf, xf = get_fft(data)
abs_yf = np.abs(yf)
sort_abs_yf = sorted(abs_yf)
xf = rfftfreq(n,data_step)
thresh  = abs_yf.mean()
indices = abs_yf<thresh
yf_clean = indices*yf
plt.plot(xf[1:],np.abs(yf_clean)[1:])
plt.show()
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(0,len(sort_abs_yf),500):    
    thresh = sort_abs_yf[i]
    indices = abs_yf<thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()
#Experiment :
#Let us run a loop to produce plots and see 
# for i in range(50,500,50):    
#     thresh = sort_abs_yf[i]
#     indices = abs_yf>=thresh
#     yf_clean = indices*yf
#     new_f_clean = irfft(yf_clean)
#     plt.title("Top " + str(i))
#     plt.plot(time, data, color="b", alpha=0.5, label='original signal')
#     plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
#     plt.legend()
#     plt.show()

In [ ]:
from scipy.fft import rfft, rfftfreq
import numpy as np
import statistics

In [ ]:
curr_files = get_files("cubic","35","500")
data, time = get_window(curr_files[0], "n")

In [ ]:
len(data)

In [ ]:
len(time)

In [ ]:
def get_fft(data):
    n = len(data)
    data_step = 0.00001
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    plt.plot(xf, np.abs(yf))
    return yf,xf

In [ ]:
yf, xf = get_fft(data)

In [ ]:
abs_yf = np.abs(yf)
sort_abs_yf = sorted(abs_yf, reverse=True)
xf = rfftfreq(n,data_step)

In [ ]:
sort_abs_yf[0:10]

In [ ]:
mean = statistics.mean(sort_abs_yf)
mean

In [ ]:
indices = abs_yf>mean
yf_clean = indices*yf

In [ ]:
plt.plot(xf[1:],np.abs(yf_clean)[1:])

In [ ]:
from scipy.fft import irfft

In [ ]:
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(10,50,10):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(50,500,50):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()

In [ ]:
curr_files = get_files("reno","35","500")
data, time = get_window(curr_files[0], "n")
print(len(data), len(time))
yf, xf = get_fft(data)
print(len(yf), len(xf))
abs_yf = np.abs(yf)
sort_abs_yf = sorted(abs_yf, reverse=True)
mean = statistics.mean(sort_abs_yf)
indices = abs_yf>mean
yf_clean = indices*yf

In [ ]:
plt.plot(xf[1:],np.abs(yf_clean)[1:])

In [ ]:
thresh = mean
indices = abs_yf>=thresh
yf_clean = indices*yf
new_f_clean = irfft(yf_clean)
plt.title("Mean")
plt.plot(time[1:], new_f_clean)
plt.show()

In [ ]:
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(10,50,10):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(50,500,50):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time[1:], new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()

In [ ]:
curr_files = get_files("bbr","35","500")
data, time = get_window(curr_files[0], "n")
print(len(data), len(time))
yf, xf = get_fft(data)
plt.show()
print(len(yf), len(xf))
abs_yf = np.abs(yf)
sort_abs_yf = sorted(abs_yf, reverse=True)
mean = statistics.mean(sort_abs_yf)
indices = abs_yf>mean
yf_clean = indices*yf
plt.title("Greater than Mean")
plt.plot(xf[1:],np.abs(yf_clean)[1:])

In [ ]:
thresh = mean
indices = abs_yf>=thresh
yf_clean = indices*yf
new_f_clean = irfft(yf_clean)
plt.title("Mean")
plt.plot(time, new_f_clean)
plt.show()

In [ ]:
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(10,50,10):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time, new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()
#Experiment :
#Let us run a loop to produce plots and see 
for i in range(50,500,50):    
    thresh = sort_abs_yf[i]
    indices = abs_yf>=thresh
    yf_clean = indices*yf
    new_f_clean = irfft(yf_clean)
    plt.title("Top " + str(i))
    plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    plt.plot(time, new_f_clean, 'k', label='FFT smoothing}', linewidth=2)
    plt.legend()
    plt.show()

In [ ]:
plt.plot(time, data)

# Using wavelet transform: How is it different from Fourier. 
#### Fourier works well when the frequency spectrum is stationary. As in you take the signal from any time to any time, the frequency spectrum would always remain the same. However, if you can see here, the peaks occur regularly but not exactly at regular intervals. Plus the first peak is different from other peaks. In these cases wavelet transform is used. It basically introduces another feature of time to work with in the freqeuncy spectrum of a signal.(Motivation to do this is 2 fold : Get a better filter, or get a frequency pattern, through which maybe classification becomes easier). So we can use a DWT filter.

In [ ]:
import pywt

In [ ]:
signal = data

In [ ]:
import pywt
signal = data
def lowpassfilter(signal, thresh=0.63, wavelet="db4"):
    thresh = thresh*np.nanmax(signal)
    coeff = pywt.wavedec(signal, wavelet, mode="per")
    coeff[1:] = (pywt.threshold(i, value=thresh, mode="hard") for i in coeff[1:])
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode="per")
    return reconstructed_signal
def plot_dwt(thresh,wv="db4"):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(signal, color="b", alpha=0.5, label='original signal')
    rec = lowpassfilter(signal, 0.5, wv)
    ax.plot(rec, 'k', label='DWT smoothing}', linewidth=2)
    ax.legend()
    ax.set_title('Removing High Frequency Noise with DWT', fontsize=18)
    ax.set_ylabel('Signal Amplitude', fontsize=16)
    ax.set_xlabel('Sample No', fontsize=16)
    plt.show()
plot_dwt(0.1,"rbio1.5")

In [ ]:
def plot_dwt(thresh,wv="db4"):
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(signal, color="b", alpha=0.5, label='original signal')
    rec = lowpassfilter(signal, 0.5, wv)
    ax.plot(rec, 'k', label='DWT smoothing}', linewidth=2)
    ax.legend()
    ax.set_title('Removing High Frequency Noise with DWT', fontsize=18)
    ax.set_ylabel('Signal Amplitude', fontsize=16)
    ax.set_xlabel('Sample No', fontsize=16)
    plt.show()

In [ ]:
plot_dwt(0.1,"rbio1.5")

In [ ]:
plot_dwt(0.5,"db3")

In [ ]:
plot_dwt(0.5,"sym7")

## This sym7 is better than the other results.


# Getting the sites 

In [ ]:
file = "websites.txt"

In [ ]:
with open(file) as f :
    lines = f.readlines()
for line in range(len(lines)):
    if(line not in  [2,3,5,7,17,18,19,25]):
        name = lines[line][:-5]
        print(name.lower(), end=" ")